In [1]:
!pip install tensorflow pandas scikit-learn



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping

pd.set_option('display.max_columns',None)

In [3]:
def call_rmsle(y_true, y_pred):
    # Ensure predictions and true values are positive by adding 1
    y_true = tf.math.maximum(y_true, 0)
    y_pred = tf.math.maximum(y_pred, 0)

    # Compute the logarithm of the true and predicted values
    log_true = tf.math.log1p(y_true)
    log_pred = tf.math.log1p(y_pred)

    # Compute the squared difference
    squared_diff = tf.math.square(log_true - log_pred)

    # Compute the mean of the squared differences
    mean_squared_diff = tf.math.reduce_mean(squared_diff)

    # Return the square root of the mean squared differences
    return tf.math.sqrt(mean_squared_diff)

def remove_leading_zeros(df):
    # Find the index of the first non-zero sales value
    first_non_zero_index = df['sales'].ne(0).idxmax()

    # Filter the DataFrame to remove leading zeros
    return df.loc[first_non_zero_index:]

In [4]:
df_train_all    = pd.read_csv('data/RELIANCE.NS.csv')


In [5]:
df_train_all.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1232,13-08-2024,2921.500000,2940.149902,2916.000000,2927.250000,2917.348633,3132532
1233,14-08-2024,2927.100098,2944.000000,2907.399902,2923.699951,2913.810547,3133733
1234,16-08-2024,2937.300049,2960.800049,2916.100098,2956.399902,2946.399902,4708451
1235,19-08-2024,2965.000000,2996.600098,2961.000000,2987.699951,2987.699951,3101541
1236,20-08-2024,2994.450000,3003.650000,2985.000000,0.000000,0.000000,1159552


In [6]:
df_train_all['10DMA'] =  df_train_all['Adj Close'].rolling(window=10).mean()
df_train_all['30DMA'] =  df_train_all['Adj Close'].rolling(window=30).mean()
df_train_all['100DMA'] =  df_train_all['Adj Close'].rolling(window=100).mean()


In [7]:
df_train_all.dropna(subset=['100DMA'], inplace=True)


In [8]:
import calendar
from datetime import datetime, timedelta
def is_last_thursday(date):
    year = date.year
    month = date.month
    
    last_day = calendar.monthrange(year, month)[1]
    last_date = datetime(year, month, last_day)
    last_day_weekday = last_date.weekday()
    days_since_last_thursday = (last_day_weekday - calendar.THURSDAY) % 7
    last_thursday_date = last_date - timedelta(days=days_since_last_thursday)
    
    return 1 if date.date() == last_thursday_date.date() else 0




In [9]:
df_train_all['Date'] = pd.to_datetime(df_train_all['Date'])

df_train_all['day'] = df_train_all['Date'].dt.day
df_train_all['month'] = df_train_all['Date'].dt.month
df_train_all['year'] = df_train_all['Date'].dt.year

df_train_all['opt_exp'] = df_train_all['Date'].apply(is_last_thursday)

C:\Users\Monil\AppData\Local\Temp\ipykernel_32972\1557585793.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_train_all['Date'] = pd.to_datetime(df_train_all['Date'])


In [10]:
df_train_all['year'] = df_train_all['year'] - df_train_all['year'].min()


# Convert month and day to cyclical features
#df_train_all['month_sin'] = np.sin(2 * np.pi * df_train_all['month'] / 12)
#df_train_all['month_cos'] = np.cos(2 * np.pi * df_train_all['month'] / 12)
#df_train_all['day_sin'] = np.sin(2 * np.pi * df_train_all['day'] / 31)
#df_train_all['day_cos'] = np.cos(2 * np.pi * df_train_all['day'] / 31)



In [11]:
#df_train_all['price_next'] = df_train_all['Close'].shift(-1)
df_train_all['pct_change'] = df_train_all['Close']/df_train_all['Open']  
df_train_all['prev_close'] = df_train_all['Close'].shift(1)
df_train_all['opening_gap'] = df_train_all['Open'] - df_train_all['prev_close'] 
df_train_all['daily_move'] = df_train_all['High'] - df_train_all['Low']
df_train_all['daily_move']= df_train_all['daily_move'].shift(1) #previous day daily movement
df_train_all['daily_move_avg'] =  df_train_all['daily_move'].rolling(window=1).mean() #average of last 5 day movement
#df_train_all['price_lag2'] = df_train_all['Adj Close'].shift(2)
#df_train_all['price_lag3'] = df_train_all['Adj Close'].shift(3)
#df_train_all['price_lag4'] = df_train_all['Adj Close'].shift(4)
#df_train_all['price_lag5'] = df_train_all['Adj Close'].shift(5)

In [12]:
#df_train_all.dropna(subset=['price_lag5'], inplace=True)

In [13]:
df_train_all.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,10DMA,30DMA,100DMA,day,month,year,opt_exp,pct_change,prev_close,opening_gap,daily_move,daily_move_avg
99,2020-01-14,1408.072388,1417.215698,1391.477295,1398.380493,1373.828979,7908267,1373.473962,1393.036979,1274.930900,14,1,0,0,0.993117,NaN,NaN,NaN,NaN
100,2020-01-15,1404.277954,1407.980957,1388.185669,1393.305908,1368.843506,7908929,1374.754028,1391.388643,1277.008174,15,1,0,0,0.992187,1398.380493,5.897461,25.738403,25.738403
101,2020-01-16,1398.014771,1411.135376,1397.100342,1406.152222,1381.464233,6423987,1374.987585,1390.945492,1279.361217,16,1,0,0,1.005821,1393.305908,4.708863,19.795288,19.795288
102,2020-01-17,1420.415894,1449.171631,1420.141602,1445.560059,1420.180054,14731735,1378.926550,1391.848263,1282.146333,17,1,0,0,1.017702,1406.152222,14.263672,14.035034,14.035034
103,2020-01-20,1471.161377,1471.161377,1395.637451,1401.077759,1376.478760,16272925,1381.697754,1391.173063,1284.711819,20,1,0,0,0.952362,1445.560059,25.601318,29.030029,29.030029
104,2020-01-21,1397.649048,1413.421265,1391.614380,1402.494995,1377.871216,9461360,1382.533154,1390.014290,1287.029832,21,1,0,0,1.003467,1401.077759,-3.428711,75.523926,75.523926
105,2020-01-22,1411.729736,1414.244141,1399.934814,1401.992065,1377.376953,5161408,1384.347681,1389.157926,1289.424196,22,1,0,0,0.993102,1402.494995,9.234741,21.806885,21.806885
106,2020-01-23,1404.872192,1409.855347,1390.425781,1396.048950,1371.538208,5623869,1382.447839,1388.093465,1291.687861,23,1,0,0,0.993720,1401.992065,2.880127,14.309327,14.309327
107,2020-01-24,1396.186035,1404.735107,1388.459961,1391.203003,1366.777222,7314222,1380.103332,1386.696639,1294.007667,24,1,0,0,0.996431,1396.048950,0.137085,19.429566,19.429566
108,2020-01-27,1385.122681,1393.854492,1376.070679,1377.487915,1353.303101,6693875,1376.766223,1384.410518,1296.386312,27,1,0,0,0.994488,1391.203003,-6.080322,16.275146,16.275146


In [14]:
df_train_all.drop(['High','Low','Adj Close','daily_move','prev_close'],axis =1, inplace = True) 

In [15]:
df_train_all.tail()

,Date,Open,Close,Volume,10DMA,30DMA,100DMA,day,month,year,opt_exp,pct_change,opening_gap,daily_move_avg
1232,2024-08-13,2921.500000,2927.250000,3132532,2937.216138,3041.889233,2946.370569,13,8,4,0,1.001968,0.250000,30.500000
1233,2024-08-14,2927.100098,2923.699951,3133733,2928.530615,3035.024194,2947.240117,14,8,4,0,0.998838,-0.149902,24.149902
1234,2024-08-16,2937.300049,2956.399902,4708451,2921.135693,3030.092594,2948.011997,16,8,4,0,1.006503,13.600098,36.600098
1235,2024-08-19,2965.000000,2987.699951,3101541,2921.055005,3026.431356,2949.480415,19,8,4,0,1.007656,8.600098,44.699951
1236,2024-08-20,2994.450000,0.000000,1159552,2632.569140,2920.881258,2920.703086,20,8,4,0,0.000000,6.750049,35.600098


In [16]:
df_train_all.dropna(subset=['daily_move_avg'], inplace=True)


In [17]:
# df = df_train_all.copy()

In [18]:
# # Create lists to hold the new columns
# dates = []
# price_type = []
# prices = []

# # Populate the lists using iloc
# for i in range(len(df)):
#     dates.append(df.iloc[i]['Date'])
#     price_type.append('Open')
#     prices.append(df.iloc[i]['Open'])
    
#     dates.append(df.iloc[i]['Date'])
#     price_type.append('Close')
#     prices.append(df.iloc[i]['Close'])

# # Creating a new DataFrame with the desired format
# new_df = pd.DataFrame({
#     'Date': dates,
#     'Type': price_type,
#     'Price': prices
# })

# # Showing the result
# print(new_df.head())


        Date   Type        Price
0 2020-01-15   Open  1404.277954
1 2020-01-15  Close  1393.305908
2 2020-01-16   Open  1398.014771
3 2020-01-16  Close  1406.152222
4 2020-01-17   Open  1420.415894


In [19]:
df_train = df_train_all[df_train_all['Date']<=  '2024-07-31']
df_test  = df_train_all[df_train_all['Date']>  '2024-07-31']

In [20]:
df_lstm_results  = pd.DataFrame()

In [21]:
df_test.tail()

,Date,Open,Close,Volume,10DMA,30DMA,100DMA,day,month,year,opt_exp,pct_change,opening_gap,daily_move_avg
1232,2024-08-13,2921.500000,2927.250000,3132532,2937.216138,3041.889233,2946.370569,13,8,4,0,1.001968,0.250000,30.500000
1233,2024-08-14,2927.100098,2923.699951,3133733,2928.530615,3035.024194,2947.240117,14,8,4,0,0.998838,-0.149902,24.149902
1234,2024-08-16,2937.300049,2956.399902,4708451,2921.135693,3030.092594,2948.011997,16,8,4,0,1.006503,13.600098,36.600098
1235,2024-08-19,2965.000000,2987.699951,3101541,2921.055005,3026.431356,2949.480415,19,8,4,0,1.007656,8.600098,44.699951
1236,2024-08-20,2994.450000,0.000000,1159552,2632.569140,2920.881258,2920.703086,20,8,4,0,0.000000,6.750049,35.600098


In [22]:
          
df = df_train.copy()
df.drop('Date',axis = 1, inplace = True)
###########################################################################################
###############     Preparing for LSTM              ######################################
#########################################################################################

# Features to be scaled
features_to_scale = ['Open', 'Close','Volume','10DMA','year','day','month','opening_gap','daily_move_avg']
#features_to_scale = ['Open', 'Close','Volume','10DMA','30DMA','100DMA','opening_gap','daily_move_avg']

# Features to be left unscaled (those ending with 'sin' or 'cos')
#features_to_leave = ['opt_exp','month_sin','month_cos','day_sin','day_cos' ]
features_to_leave = ['opt_exp' ]

features = features_to_scale + features_to_leave
#target = 'price_next'
target = 'pct_change'

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, shuffle=False)

#X_train = df[features]
#y_train = df[target]
# Initialize scalers
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale only the selected features
X_train_scaled_part = feature_scaler.fit_transform(X_train[features_to_scale])
X_test_scaled_part = feature_scaler.transform(X_test[features_to_scale])

# Combine scaled features with unscaled features (still as NumPy arrays)
X_train_scaled = np.hstack((X_train_scaled_part, X_train[features_to_leave].values))
X_test_scaled = np.hstack((X_test_scaled_part, X_test[features_to_leave].values))

# Scale target variable
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))



# Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
timesteps = 1  # You need to define this based on your data and model requirements
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))



In [23]:

# Build your LSTM model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
#model.add(LSTM(256))
#model.add(Dropout(0.2))

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model with RMSLE as the loss function
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss=call_rmsle )

# Train the model
history = model.fit(X_train_reshaped, y_train_scaled, epochs=100, 
                    validation_data=(X_test_reshaped, y_test_scaled), batch_size=64,callbacks=[early_stopping])


C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 256)

In [ ]:

##############################################################################################
#########################          Make Predictions              #############################
##############################################################################################


# Predictions
lstm_predictions = model.predict(X_test_reshaped)

# Ensure both predictions and actual values are of type float64
y_test_scaled = y_test_scaled.reshape(-1, 1).astype('float64')
lstm_predictions_scaled = lstm_predictions.astype('float64')

# Calculate RMSLE
lstm_rmsle_val = call_rmsle(y_test_scaled, lstm_predictions_scaled)

y_test_actual = target_scaler.inverse_transform(y_test_scaled).astype('float64')
lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')
lstm_rmsle_val_actual = call_rmsle(y_test_actual, lstm_predictions_actual)
print("RSMLE we got is ",lstm_rmsle_val_actual)

#

## Applying model on Test Data

In [ ]:
#############################################################################################
#########################             Applying model on test data       ######################
##############################################################################################

del df
df   = df_test.copy()

df.drop('Date',axis = 1, inplace = True)
###########################################################################################
###############     Preparing for LSTM              ######################################
#########################################################################################


# Scale only the selected features
X_test_scaled_part = feature_scaler.transform(df[features_to_scale])

# Combine scaled features with unscaled features (still as NumPy arrays)
X_test_scaled = np.hstack((X_test_scaled_part, df[features_to_leave].values))

X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

# Predictions
lstm_predictions = model.predict(X_test_reshaped)
lstm_predictions_scaled = lstm_predictions.astype('float64')

lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')

# Add predictions to the dataframe
df_test['pct_change_predicted'] = lstm_predictions_actual



In [ ]:
#df_test['predicted_price_tomorrow'] = df_test['pct_change_predicted']*df_test['Close']
#df_test['predicted_price_today'] = df_test['predicted_price_tomorrow'].shift(1)
df_test['predicted_close'] = df_test['Open']*df_test['pct_change_predicted']

In [ ]:
df_test.head(100)

In [ ]:
df_test.dropna(subset=['predicted_close'], inplace=True)

In [ ]:
df_test[['Close','predicted_close']]

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual prices ('Adj Close') and the predicted prices ('predicted_price')
plt.plot(df_test['Close'], label='Adj Close')
plt.plot(df_test['predicted_close'], label='Predicted Price')

# Add a title, labels, and legend
plt.title('Actual vs Predicted Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()

# Display the plot
plt.show()


In [ ]:
#df_lstm_results.to_csv('data_dir/output_data/lstm_prediction_submission_47_54.csv',index=False)